In [44]:
!pip install tifffile

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
### This script reads input images and output label definitions, and it creates a single data matrix with all images and a label mask image
### Details:
### - Input images are panchromatic (1 channel) and multispectral (8 channels);
### - Multispectral images are resized to panchromatic image size (higher resolution)
### - Class labels for each image/object type are read from the list of polygons (train_wkt_v4.csv)
### - They are used to create a single label mask (0: no object, 1-11: different objects)
### - Output matrix included [panchromatic image, multispectral images, class label mask], and is saved as a pickle file


import sys
import cv2
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
import numpy as np
import pandas as pd
import tifffile as tiff
import os
from skimage.transform import resize
import pickle




In [0]:
############################################
## Functions

## Create a binary mask from a given polygon
def polyToMask(imgSize, polygons):
    labelMask = np.zeros(imgSize, np.uint16)
    if not polygons:
        print('    EMPTY poly object')
        return labelMask
    int_coords = lambda x: np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
    interiors = [int_coords(pi.coords) for poly in polygons for pi in poly.interiors]
    cv2.fillPoly(labelMask, exteriors, 1)
    cv2.fillPoly(labelMask, interiors, 0)
    return labelMask


## Calculate scaling in x and y
def getScaleXY(imgSize, xMax, yMin):
    h, w = imgSize
    wc = w * (w / (w + 1))
    hc = h * (h / (h + 1))
    return wc / xMax, hc / yMin


def scaleImg(imgIn, percLow, percHigh):
    w, h, d = imgIn.shape
    imgOut = np.reshape(imgIn, [w * h, d]).astype(np.float64)
    mins = np.percentile(imgOut, percLow, axis=0)
    maxs = np.percentile(imgOut, percHigh, axis=0)
    difs = maxs - mins
    imgOut = (imgOut - mins[None, :]) / difs[None, :]
    imgOut = np.reshape(imgOut, [w, h, d])
    imgOut = imgOut.clip(0, 1)
    imgOut = (imgOut * 255).round().astype(np.uint16)
    return imgOut



In [0]:
############################################
## Hard coded paths and vars
BDIR = '/content/drive/My Drive/PrjData/Prj_DSTL/Data_Init'
IMGDIR = BDIR + '/Images_SmallSampleSize'
IMGLIST = IMGDIR + '/ListID.csv'  
tfile = BDIR + '/' + 'train_wkt_v4.csv'
gfile = BDIR + '/' + 'grid_sizes.csv'

odir = '/content/drive/My Drive/PrjData/Prj_DSTL/Data_Prep'

In [0]:
## Create output dir
if os.path.exists(odir)==False:
  os.mkdir(odir)

In [50]:
############################################
## Main

## Read csv files
dfID = pd.read_csv(IMGLIST)
dfP = pd.read_csv(tfile)
dfG = pd.read_csv(gfile, names=['ImageId', 'Xmax', 'Ymin'], skiprows=1)

## For each image in the list create a mask with labels of all objects
imgIds = dfID.IMG_ID.tolist()

ListImgAll = []
ListLabelAll = []
for i, tmpId in enumerate(imgIds):

    print('\nImage no ' + str(i) + ': ' + tmpId)
    
    fout = odir + '/' + tmpId + '_allChannelsAndLabels.p'

    if os.path.exists(fout + 'ee')==True:
        print('   Out mat exists, skip ...')
    
    else:

        #### Read panchromatic image
        imgPanchr = tiff.imread(IMGDIR + '/' + tmpId + '_P.tif')
        imgSize = imgPanchr.shape
        imgPanchr = imgPanchr.reshape([imgSize[0], imgSize[1], 1])

        #### Read multispectral images, resize to the size of panchromatic images
        imgMSpec = tiff.imread(IMGDIR + '/' + tmpId + '_M.tif').transpose([1, 2, 0])
        imgMSpecUp = np.zeros([imgSize[0], imgSize[1], imgMSpec.shape[2]]).astype(np.uint16)
        for i in np.arange(0, imgMSpec.shape[2]):
            imgMSpecUp[:,:,i] = resize(imgMSpec[:,:,i].astype(np.float), (imgSize[0], imgSize[1])).round()
        imgMSpecUp = imgMSpecUp.astype(np.uint16)

        #### Read multispectral images, resize to the size of panchromatic images
        imgSWIR = tiff.imread(IMGDIR + '/' + tmpId + '_A.tif').transpose([1, 2, 0])
        imgSWIRUp = np.zeros([imgSize[0], imgSize[1], imgSWIR.shape[2]]).astype(np.uint16)
        for i in np.arange(0, imgSWIR.shape[2]):
            imgSWIRUp[:,:,i] = resize(imgSWIR[:,:,i].astype(np.float), (imgSize[0], imgSize[1])).round()
        imgSWIRUp = imgSWIRUp.astype(np.uint16)

        #### Create mask of all objects
        # Read grid size
        xMax = dfG[dfG.ImageId == tmpId].Xmax
        yMin = dfG[dfG.ImageId == tmpId].Ymin
        scX, scY = getScaleXY(imgSize, xMax, yMin)
        
        maskAll = np.zeros([imgSize[0], imgSize[1], 11], np.uint16)
        dfSel = dfP[dfP.ImageId == tmpId]
        
        # Create mask for each object (polygon with coordinates)
        for j, tmpRow in dfSel.iterrows():      
            print(' Object no ' + str(j) + ': ' + str(tmpRow.ClassType))
            polyInit = shapely.wkt.loads(tmpRow.MultipolygonWKT)

            # Scale polygons
            polySc = shapely.affinity.scale(polyInit, xfact=scX, yfact=scY, origin=(0, 0, 0))

            # Add mask image for the object
            maskAll[:,:,tmpRow.ClassType] = polyToMask(imgSize, polySc)
            
        # Select the class type in each pixel
        maskComb = np.argmax(maskAll, axis=2).astype(np.uint16).reshape([imgSize[0], imgSize[1], 1])

        #### Create img matrix from all channels and label mask
        outMat = np.concatenate((imgPanchr, imgMSpecUp, maskComb), axis=2).astype(np.uint16)     

        ### Write out image and label mat
        print('Writing images to: ' + fout)
        pickle.dump(outMat, open(fout, 'wb'))



Image no 0: 6040_2_2
 Object no 0: 1
    EMPTY poly object
 Object no 1: 2
    EMPTY poly object
 Object no 2: 3
    EMPTY poly object
 Object no 3: 4
 Object no 4: 5
 Object no 5: 6
    EMPTY poly object
 Object no 6: 7
    EMPTY poly object
 Object no 7: 8
    EMPTY poly object
 Object no 8: 9
    EMPTY poly object
 Object no 9: 10
    EMPTY poly object
Writing images to: /content/drive/My Drive/PrjData/Prj_DSTL/Data_Prep/6040_2_2_allChannelsAndLabels.p

Image no 1: 6070_2_3
 Object no 210: 1
 Object no 211: 2
 Object no 212: 3
 Object no 213: 4
    EMPTY poly object
 Object no 214: 5
 Object no 215: 6
    EMPTY poly object
 Object no 216: 7
 Object no 217: 8
 Object no 218: 9
    EMPTY poly object
 Object no 219: 10
Writing images to: /content/drive/My Drive/PrjData/Prj_DSTL/Data_Prep/6070_2_3_allChannelsAndLabels.p

Image no 2: 6100_2_3
 Object no 110: 1
 Object no 111: 2
 Object no 112: 3
 Object no 113: 4
 Object no 114: 5
 Object no 115: 6
    EMPTY poly object
 Object no 116: 

In [51]:
ls  '/content/drive/My Drive/PrjData/Prj_DSTL/Data_Prep'

6040_2_2_allChannelsAndLabels.p  6100_2_3_allChannelsAndLabels.p
6070_2_3_allChannelsAndLabels.p  6150_2_3_allChannelsAndLabels.p
